In [54]:
import open3d as o3d
import numpy as np
from copy import deepcopy
import trimesh
import subprocess
import os
import time

In [3]:
static_parts = o3d.io.read_triangle_mesh(fr"G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0001.obj")
static_parts.compute_vertex_normals()

TriangleMesh with 71221 points and 139834 triangles.

In [4]:
dynamic_deformed = o3d.io.read_triangle_mesh(r"G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\drinking\reference/deformed_reference_mesh_007.obj")
dynamic_deformed.compute_vertex_normals()
dynamic_deformed.paint_uniform_color([1, 0, 0])

TriangleMesh with 6524 points and 8999 triangles.

In [9]:
dynamic_deformed_meshes = []
for i in range(1, 8):
    dynamic_deformed = o3d.io.read_triangle_mesh(fr"G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\drinking\reference/deformed_reference_mesh_{i:03}.obj")
    dynamic_deformed.compute_vertex_normals()
    dynamic_deformed.paint_uniform_color([1, 0, 0])
    dynamic_deformed_meshes.append(dynamic_deformed)

In [10]:
dynamic_meshes = []
for i in range(1, 8):
    dynamic = o3d.io.read_triangle_mesh(fr"G:\VS2022Projects\arap-volume-tracking-main\data\scene\Dynamic/dynamic_mesh_0{i:03}.obj")
    dynamic.compute_vertex_normals()
    dynamic.paint_uniform_color([0.4, 0.4, 0.4])
    dynamic_meshes.append(dynamic)

In [11]:
o3d.visualization.draw_geometries([dynamic_deformed_meshes[6], dynamic_meshes[6]])

In [12]:
def subdivide_surface_fitting(decimated_mesh, target_mesh, iterations=1):
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decimated_mesh, number_of_iterations=iterations)
    print(subdivided_mesh)
    subdivided_mesh.compute_vertex_normals()
    
    pcd_target = o3d.geometry.PointCloud()
    pcd_target.points = o3d.utility.Vector3dVector(target_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_target)
    subdivided_vertices = np.array(subdivided_mesh.vertices)
    target_vertices = np.array(target_mesh.vertices)
    fitting_vertices = deepcopy(subdivided_vertices)
    
    for i in range(0, len(subdivided_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_vertices[i], 1)
        fitting_vertices[i] = target_vertices[np.asarray(index)]
        
    subdivided_mesh.vertices = o3d.utility.Vector3dVector(fitting_vertices)
    return subdivided_mesh

In [20]:
def read_triangle_mesh_with_trimesh(avatar_name,enable_post_processing=False):
    # EDIT: next 4 lines replace to maintain order even in case of degenerate and non referenced
    # scene_patch = trimesh.load(avatar_name,process=enable_post_processing)
    if enable_post_processing:
        scene_patch = trimesh.load(avatar_name,process=True)
    else:
        scene_patch = trimesh.load(avatar_name,process=False,maintain_order=True) 
    mesh = o3d.geometry.TriangleMesh(
        o3d.utility.Vector3dVector(scene_patch.vertices),
        o3d.utility.Vector3iVector(scene_patch.faces)
    ) 
    if scene_patch.vertex_normals.size:
        mesh.vertex_normals = o3d.utility.Vector3dVector(scene_patch.vertex_normals.copy())
    if scene_patch.visual.defined:
        # either texture or vertex colors if no uvs present.
        if scene_patch.visual.kind == 'vertex':
            mesh.vertex_colors = o3d.utility.Vector3dVector(scene_patch.visual.vertex_colors[:,:3]/255) # no alpha channel support
        elif scene_patch.visual.kind == 'texture':
            uv = scene_patch.visual.uv
            if uv.shape[0] == scene_patch.vertices.shape[0]:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv[scene_patch.faces.flatten()])
            elif uv.shape[0] != scene_patch.faces.shape[0] * 3:
                assert False
            else:
                mesh.triangle_uvs = o3d.utility.Vector2dVector(uv)
                if scene_patch.visual.material is not None and scene_patch.visual.material.image is not None:
                    if scene_patch.visual.material.image.mode == 'RGB':
                        mesh.textures = [o3d.geometry.Image(np.asarray(scene_patch.visual.material.image))]
                    else:
                        assert False
        else:
            assert False
    return mesh

In [71]:
def compute_D1_psnr(original_mesh, decoded_mesh):
    
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.linalg.norm(original_vertices[i] - decoded_vertices[index]))
    MSE = MSE / len(original_vertices)
    #print("D1 mse:",MSE)
    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    #print(signal_peak)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    #print(psnr)
    return psnr

def compute_D2_psnr(original_mesh, decoded_mesh):
    
    decoded_mesh.compute_vertex_normals()
    original_vertices = np.array(original_mesh.vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_decoded.normals = o3d.utility.Vector3dVector(decoded_mesh.vertex_normals)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.dot((original_vertices[i] - decoded_vertices[index])[0], np.array(pcd_decoded.normals)[index][0]))
    MSE = MSE / len(original_vertices)

    aabb = pcd_original.get_axis_aligned_bounding_box()
    min_bound = aabb.get_min_bound()

    max_bound = aabb.get_max_bound()

    signal_peak = np.linalg.norm(max_bound - min_bound)
    psnr = 20 * np.log10(signal_peak) - 10 * np.log10(MSE)
    
    return psnr

def compute_MSE_RMSE(original_mesh, decoded_mesh):
    
    original_vertices = np.array(original_mesh.vertices)
    #original_vertices = normalize_vertices(original_vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    #decoded_vertices = normalize_vertices(decoded_vertices)
    
    pcd_original = o3d.geometry.PointCloud()
    pcd_original.points = o3d.utility.Vector3dVector(original_vertices)
    
    pcd_decoded = o3d.geometry.PointCloud()
    pcd_decoded.points = o3d.utility.Vector3dVector(decoded_vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_decoded)
    
    MSE = 0
    for i in range(0, len(original_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(original_vertices[i], 1)
        MSE += np.square(np.linalg.norm(original_vertices[i] - decoded_vertices[index]))
    MSE = MSE / len(original_vertices)
    #print("MSE:", MSE)
    RMSE =np.sqrt(MSE)
    
    return np.log10(MSE), np.log10(RMSE)

from scipy.spatial.distance import directed_hausdorff
def compute_Hausdorff(original_mesh, decoded_mesh):
    original_vertices = np.array(original_mesh.vertices)
    decoded_vertices = np.array(decoded_mesh.vertices)
    hausdorff = directed_hausdorff(original_vertices, decoded_vertices)
    return hausdorff[0] * 1e4


In [24]:
for i in range(1, 8):
    dynamic_deformed = o3d.io.read_triangle_mesh(fr"G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\drinking\reference/deformed_reference_mesh_{i:03}.obj")
    original_i = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene/mesh_000{i:1}.obj')

    print(dynamic_deformed)
    print(original_i)
    dynamic_deformed.compute_vertex_normals()
    original_i.compute_vertex_normals()
    #o3d.visualization.draw_geometries([reconstruct_dancer_i])
    fitting_mesh_dancer_i = subdivide_surface_fitting(dynamic_deformed, original_i, 1)
    #print(np.array(fitting_mesh_dancer_i.triangles))
    o3d.io.write_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\drinking\reference/fitting_mesh_{i:03}.obj', fitting_mesh_dancer_i, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    #o3d.visualization.draw_geometries([fitting_mesh_dancer_i])

TriangleMesh with 6555 points and 9000 triangles.
TriangleMesh with 84034 points and 168589 triangles.
TriangleMesh with 22141 points and 36000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 6555 points and 9000 triangles.
TriangleMesh with 83905 points and 168328 triangles.
TriangleMesh with 22141 points and 36000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 6555 points and 9000 triangles.
TriangleMesh with 83881 points and 168301 triangles.
TriangleMesh with 22141 points and 36000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 6555 points and 9000 triangles.
TriangleMesh with 82407 points and 165406 triangles.
TriangleMesh with 22141 points and 36000 triangles.
[Open3D WARNING] Write OBJ can not include triangle normals.
TriangleMesh with 6555 points and 9000 triangles.
TriangleMesh with 81981 points and 164452 triangles.
TriangleMesh with 22141 points an

In [18]:
loaded_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
print(loaded_decimated_reference_mesh)
loaded_decimated_reference_mesh_vertices = np.array(loaded_decimated_reference_mesh.vertices)
print(np.array(loaded_decimated_reference_mesh.triangles))
loaded_decimated_reference_mesh_vertices

TriangleMesh with 6555 points and 9000 triangles.
[[   0    1    2]
 [   3    4    5]
 [   0    2    6]
 ...
 [3680 3583 3661]
 [3654 3644 3661]
 [3684 3654 3661]]


array([[-0.684466  ,  1.83513999, -0.88690299],
       [-0.67031801,  1.80734003, -0.89626801],
       [-0.65697098,  1.82257009, -0.89857602],
       ...,
       [-0.69024301,  1.86600995,  2.08393002],
       [-0.25770801,  2.20842004, -0.684578  ],
       [-0.16192999,  0.86249697,  0.26020899]])

In [21]:
dataset = "drinking"

In [25]:
subdivided_decimated_reference_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(loaded_decimated_reference_mesh, number_of_iterations=1)
#print(subdivided_decimated_reference_mesh)
subdivided_decimated_reference_mesh_vertices = np.array(subdivided_decimated_reference_mesh.vertices)
#o3d.visualization.draw_geometries([subdivided_decimated_reference_mesh])
displacements = []
for i in range(1, 8):
    offset = 0
    fitting_mesh_dancer_i = read_triangle_mesh_with_trimesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/fitting_mesh_{i+offset:03}.obj', enable_post_processing=False)
    #fitting_mesh_dancer_i = fitting_mesh_dancer[i]
    print(fitting_mesh_dancer_i, subdivided_decimated_reference_mesh)
    fitting_mesh_vertices = np.array(fitting_mesh_dancer_i.vertices)
    #print(np.array(fitting_mesh_dancer_i.triangles))
    displacement_i = fitting_mesh_vertices - subdivided_decimated_reference_mesh_vertices
    np.savetxt(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/displacements_{dataset}_{i+offset:03}.txt', displacement_i, fmt='%8f')
    displacements.append(displacement_i)



TriangleMesh with 22141 points and 36000 triangles. TriangleMesh with 22141 points and 36000 triangles.
TriangleMesh with 22141 points and 36000 triangles. TriangleMesh with 22141 points and 36000 triangles.
TriangleMesh with 22141 points and 36000 triangles. TriangleMesh with 22141 points and 36000 triangles.
TriangleMesh with 22141 points and 36000 triangles. TriangleMesh with 22141 points and 36000 triangles.
TriangleMesh with 22141 points and 36000 triangles. TriangleMesh with 22141 points and 36000 triangles.
TriangleMesh with 22141 points and 36000 triangles. TriangleMesh with 22141 points and 36000 triangles.
TriangleMesh with 22141 points and 36000 triangles. TriangleMesh with 22141 points and 36000 triangles.


In [32]:
input_reference_mesh_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\reference_mesh/decimated_reference_mesh.obj'
input_decimated_reference_mesh = read_triangle_mesh_with_trimesh(input_reference_mesh_path, enable_post_processing=False)
print(input_decimated_reference_mesh)
np.array(input_decimated_reference_mesh.vertices)

TriangleMesh with 6555 points and 9000 triangles.


array([[-0.655131,  1.77801 , -0.898511],
       [-0.670318,  1.80734 , -0.896268],
       [-0.684466,  1.83514 , -0.886903],
       ...,
       [-0.694706,  1.69875 ,  2.09997 ],
       [-0.690243,  1.77677 ,  2.09997 ],
       [-0.690243,  1.86601 ,  2.08393 ]])

In [33]:
output_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\reference_mesh/encoded_decimated_reference_mesh.drc'
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_encoder',
                            '-i', input_reference_mesh_path,
                            '-o', output_path,
                            '-qp', str('14'),
                            '-cl', '7'
                            ], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Encoder options:
  Compression level = 7
  Positions: Quantization = 14 bits

Encoded mesh saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\reference_mesh/encoded_decimated_reference_mesh.drc (52 ms to encode).

Encoded size = 24594 bytes

For better compression, increase the compression level up to '-cl 10' .





In [38]:
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_decoder',
                            '-i', output_path,
                            '-o', fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\reference_mesh/decode_decimated_reference_mesh.obj'
                            ], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Decoded geometry saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\reference_mesh/decode_decimated_reference_mesh.obj (14 ms to decode)




In [118]:
input_static_mesh_path = fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0003.obj'
output_static_mesh_path = fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0003.drc'
result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_encoder',
                            '-i', input_static_mesh_path,
                            '-o', output_static_mesh_path,
                            '-qp', str('12'),
                            '-cl', '10'
                            ], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

result = subprocess.run([
                            r'G:\Github\draco\build\Debug\draco_decoder',
                            '-i', output_static_mesh_path,
                            '-o', fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0003_decoded.obj'
                            ], capture_output=True, text=True)
print(result.stdout)
print(result.stderr)

Encoder options:
  Compression level = 10
  Positions: Quantization = 12 bits

Encoded mesh saved to G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0003.drc (1499 ms to encode).

Encoded size = 143188 bytes



Decoded geometry saved to G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0003_decoded.obj (350 ms to decode)




In [39]:
for i in range(1, 8):
    offset = 0
    displacement = np.loadtxt(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/displacements_{dataset}_{i+offset:03}.txt')
    pcd = o3d.geometry.PointCloud()
    points = displacement
    pcd.points = o3d.utility.Vector3dVector(points)
    print(pcd)
    #o3d.io.write_point_cloud(r'G:\VS2022Projects\tvm-editing-master\TVMEditor.Test\bin\Release\net5.0\Data\Dancer\reference_mesh/dis.ply', pcd)
    points=np.asarray(pcd.points)
    dtype = o3d.core.float32
    p_tensor = o3d.core.Tensor(points, dtype=dtype)
    pc = o3d.t.geometry.PointCloud(p_tensor)
    o3d.t.io.write_point_cloud(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/dis_{dataset}_{i+offset:03}.ply', pc, write_ascii=True)

PointCloud with 22141 points.
PointCloud with 22141 points.
PointCloud with 22141 points.
PointCloud with 22141 points.
PointCloud with 22141 points.
PointCloud with 22141 points.
PointCloud with 22141 points.


In [122]:
import re

qp = 14
times = []
for i in range(1, 8):
    offset = 0
    result = subprocess.run([
                                r'G:\Github\draco\build\Debug\draco_encoder',
                                #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
                                '-point_cloud',
                                '-i', fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/dis_{dataset}_{i+offset:03}.ply',
                                '-o', fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{5}/dis_{dataset}_{i+offset:03}.drc',
                                '-qp', str(qp),
                                '-cl', '10'
                                ], capture_output=True, text=True)
    print(result.stdout)
    time_pattern = re.compile(r"\((\d+) ms to encode\)")
    match = time_pattern.search(result.stdout)
    if match:
        times.append(int(match.group(1)))
    
    result = subprocess.run([
                                r'G:\Github\draco\build\Debug\draco_encoder',
                                #r'G:\Github\draco\buildforSequenceEncoding\Debug\draco_encoder',
                                '-point_cloud',
                                '-i', fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/dis_{dataset}_{i+offset:03}.ply',
                                '-o', fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{5}/dis_{dataset}_{i+offset:03}_0.drc',
                                '-qp', str(qp),
                                '-cl', '0'
                                ], capture_output=True, text=True)
    print(result.stdout)

        
if times:
    mean_time = sum(times) / len(times)
    print(f"Mean encoding time: {mean_time:.2f} ms")
print(f"Average encoding time for qp {qp}: {mean_time:.2f} seconds\n\n")

Encoder options:
  Compression level = 10
  Positions: Quantization = 14 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\drinking\reference_mesh/GoF5/dis_drinking_001.drc (33 ms to encode).

Encoded size = 49677 bytes


Encoder options:
  Compression level = 0
  Positions: Quantization = 14 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\drinking\reference_mesh/GoF5/dis_drinking_001_0.drc (8 ms to encode).

Encoded size = 89492 bytes

For better compression, increase the compression level up to '-cl 10' .


Encoder options:
  Compression level = 10
  Positions: Quantization = 14 bits

Encoded point cloud saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\drinking\reference_mesh/GoF5/dis_drinking_002.drc (31 ms to encode).

Encoded size = 62795 bytes


Encoder options:
  Compression level = 0
  Positions: Quantization = 14 bits

Encoded point cloud s

In [116]:
times= []
for i in range(1, 8):
    offset = 0
    result = subprocess.run([
                                r'G:\Github\draco\build\Debug\draco_decoder',
                                '-i', fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{5}/dis_{dataset}_{i+offset:03}_0.drc',
                                '-o', fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{5}/decoded_{dataset}_{i+offset:03}_displacements.ply'
                                ], capture_output=True, text=True)
    print(result.stdout)
    
    result = subprocess.run([
                                r'G:\Github\draco\build\Debug\draco_decoder',
                                '-i', fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{5}/dis_{dataset}_{i+offset:03}.drc',
                                '-o', fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{5}/decoded_{dataset}_{i+offset:03}_displacements_10.ply'
                                ], capture_output=True, text=True)
    print(result.stdout)
    time_pattern = re.compile(r"\((\d+) ms to decode\)")
    match = time_pattern.search(result.stdout)
    if match:
        times.append(int(match.group(1)))
        
if times:
    mean_time = sum(times) / len(times)
    print(f"Mean encoding time: {mean_time:.2f} ms")
print(f"Average encoding time for qp {qp}: {mean_time:.2f} seconds\n\n")

Decoded geometry saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\drinking\reference_mesh/GoF5/decoded_drinking_001_displacements.ply (3 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\drinking\reference_mesh/GoF5/decoded_drinking_001_displacements_10.ply (5 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\drinking\reference_mesh/GoF5/decoded_drinking_002_displacements.ply (3 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\drinking\reference_mesh/GoF5/decoded_drinking_002_displacements_10.ply (12 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\drinking\reference_mesh/GoF5/decoded_drinking_003_displacements.ply (3 ms to decode)

Decoded geometry saved to G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.

In [43]:
def calculate_bitrate(file_size, duration):
    return file_size * 8 / duration

In [48]:
number_frames = 7
frame_rate = 30

In [123]:
total_size = 0
offset = 1
for i in range(0, 7):
    displacement_file_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{5}/dis_{dataset}_{i+offset:03}_0.drc'
    displacement_file_size = os.path.getsize(displacement_file_path)
    total_size += displacement_file_size
reference_mesh_file_path = fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/encoded_decimated_reference_mesh.drc'
reference_mesh_file_size = os.path.getsize(reference_mesh_file_path)
total_size += reference_mesh_file_size
total_duration = number_frames / frame_rate
total_size += os.path.getsize(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0003.drc')
overall_bitrate = calculate_bitrate(total_size, total_duration)

print(f"Total Size of {number_frames} DRC Files: {total_size} bytes")
print(f"Overall Bitrate: {overall_bitrate} bits per second")

bitrate_kbps = overall_bitrate / 1000
bitrate_mbps = overall_bitrate / 1000000


print(f"Overall Bitrate: {bitrate_kbps:.2f} Kbps")
print(f"Overall Bitrate: {bitrate_mbps:.2f} Mbps")

Total Size of 7 DRC Files: 798958 bytes
Overall Bitrate: 27392845.714285713 bits per second
Overall Bitrate: 27392.85 Kbps
Overall Bitrate: 27.39 Mbps


In [124]:
original_displacements = []
decoded_displacements = []
dis_plys= []
for i in range(0, 7):
    offset = 1
    original_displacement = np.loadtxt(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\output\{dataset}\reference/displacements_{dataset}_{i+offset:03}.txt')
    decoded_displacement = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{5}/decoded_{dataset}_{i+offset:03}_displacements.ply')
    dis_ply = o3d.io.read_point_cloud(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\reference_mesh/GoF{5}/decoded_{dataset}_{i+offset:03}_displacements.ply')
    original_displacements.append(original_displacement)
    decoded_displacements.append(decoded_displacement)
    dis_plys.append(dis_ply)
print(decoded_displacements.__len__())

7


In [129]:
d1s = []
d2s = []
mses = []
rmses = []
hausdorffs = []
original_static = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0003_decoded.obj', enable_post_processing=False)
for m in range(0, 7):
    decode_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\reference_mesh/decode_decimated_reference_mesh.obj', enable_post_processing=False)
    #print(decode_decimated_reference_mesh)
    np.array(decode_decimated_reference_mesh.vertices)
    start = time.time()
    subdivided_decoded_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(decode_decimated_reference_mesh, number_of_iterations=1)
    mesh = deepcopy(subdivided_decoded_mesh)
    triangles = deepcopy(mesh.triangles)
    end = time.time()
    print("subdivision time:", end - start)
    #print(subdivided_decoded_mesh)
    input_decimated_reference_mesh = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\reference_mesh/decimated_reference_mesh.obj', enable_post_processing=False)
    subdivided_mesh = o3d.geometry.TriangleMesh.subdivide_midpoint(input_decimated_reference_mesh, number_of_iterations=1)
    #print(subdivided_mesh)
    #original_static = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\{dataset}\meshes/mitch_fr0{m+offset:03}.obj')
    #original_dancer = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\tvm-editing\TVMEditor.Test\bin\Release\net5.0\Data\Drinking\meshes/dynamic_mesh_0{m+offset:03}.obj')
    original_dancer = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\meshes\good/mesh_0{m+offset:03}.obj')
    #print(original_dancer)
    decoded_mesh_vertices = np.array(decode_decimated_reference_mesh.vertices)
    subdivided_decoded_mesh_vertices = np.array(subdivided_decoded_mesh.vertices)
    
    
    displacement = np.array(decoded_displacements[m].points)
    
    dis_indexer = o3d.geometry.PointCloud()
    dis_indexer.points = o3d.utility.Vector3dVector(original_displacements[m])
    dis_tree = o3d.geometry.KDTreeFlann(dis_indexer)
    
    pcd_indexer = o3d.geometry.PointCloud()
    pcd_indexer.points = o3d.utility.Vector3dVector(subdivided_mesh.vertices)
    pcd_tree = o3d.geometry.KDTreeFlann(pcd_indexer)
    
    reordered_vertices = deepcopy(subdivided_decoded_mesh_vertices)
    start = time.time()
    for i in range(0, len(subdivided_decoded_mesh_vertices)):
        [k, index, _] = pcd_tree.search_knn_vector_3d(subdivided_decoded_mesh_vertices[i], 1)
        [j, dis_index, _] = dis_tree.search_knn_vector_3d(original_displacements[m][index[0]], 1)
        #print(displacement[dis_index], original_displacements[index[0]])
        reordered_vertices[i] += displacement[dis_index[0]]
    end = time.time()
    print("rematching time:", end - start)
    reconstruct_mesh = o3d.geometry.TriangleMesh()
    reconstruct_mesh.triangles = subdivided_decoded_mesh.triangles
    reconstruct_mesh.vertices = o3d.utility.Vector3dVector(reordered_vertices)
    reconstruct_mesh += original_static
    reconstruct_mesh.compute_vertex_normals()
    o3d.visualization.draw_geometries([reconstruct_mesh])
    o3d.io.write_triangle_mesh(fr'G:\PycharmProjects\Mesh_Editing\Results\decode_Ours\{dataset}/GoF{5}/decoded_{dataset}_fr0{m+offset:03}.obj', reconstruct_mesh, write_vertex_normals=False, write_vertex_colors=False, write_triangle_uvs=False)
    
    d1 = max(compute_D1_psnr(original_dancer, reconstruct_mesh), compute_D1_psnr(reconstruct_mesh, original_dancer))
    print("D1:", d1)
    d1s.append(d1)
    
    #d2 = max(compute_D2_psnr(original_dancer, reconstruct_mesh), compute_D2_psnr(reconstruct_mesh, original_dancer))
    #print("D2:", d2)
    #d2s.append(d2)

    #logmse1, logrmse1 = compute_MSE_RMSE(original_dancer, reconstruct_mesh)
    #logmse2, logrmse2 = compute_MSE_RMSE(reconstruct_mesh, original_dancer)
    #logmse = min(logmse1, logmse2)
    #logrmse = min(logrmse1, logrmse2)
    #print("log10 of mse:", logmse, ", log10 of rmse:", logrmse)
    #mses.append(logmse)
    #rmses.append(logrmse)
    
    #hausdorff = compute_Hausdorff(original_dancer, reconstruct_mesh)
    #print("Hausdorff distance:", hausdorff)
    #hausdorffs.append(hausdorff)
o3d.visualization.draw_geometries([reconstruct_mesh])
print("average D1:", np.mean(d1s))
#print("average D2:", np.mean(d2s))
print("average log10 of mse:", np.mean(mses))
print("average log10 of rmse:", np.mean(rmses))
print("average Hausdorff:", np.mean(hausdorffs))

subdivision time: 0.002077817916870117
rematching time: 0.15950703620910645
[Open3D WARNING] Write OBJ failed: unable to open file.
D1: 57.77863660697254
subdivision time: 0.0020072460174560547
rematching time: 0.14287900924682617
[Open3D WARNING] Write OBJ failed: unable to open file.
D1: 58.17656783742764
subdivision time: 0.001008749008178711
rematching time: 0.1721205711364746
[Open3D WARNING] Write OBJ failed: unable to open file.
D1: 80.50196015076348
subdivision time: 0.0010347366333007812
rematching time: 0.15822386741638184
[Open3D WARNING] Write OBJ failed: unable to open file.
D1: 57.70845506376931
subdivision time: 0.0010030269622802734
rematching time: 0.15407896041870117


KeyboardInterrupt: 

In [72]:
original_static = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0001.obj', enable_post_processing=False)
original_static2 = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0002.obj', enable_post_processing=False)
original_static3 = o3d.io.read_triangle_mesh(fr'G:\VS2022Projects\arap-volume-tracking-main\data\scene\Static/static_mesh_0003.obj', enable_post_processing=False)

d1 = max(compute_D1_psnr(original_static, original_static3), compute_D1_psnr(original_static3, original_static))
print("D1:", d1)
d1s.append(d1)

d2 = max(compute_D2_psnr(original_static, original_static3), compute_D2_psnr(original_static3, original_static))
print("D2:", d2)
d2s.append(d2)


D1: 56.351319582457926
D2: 66.356072542004
